# Exercise 2: Boosting

## Do not start the exercise until you fully understand the submission guidelines.


* The homework assignments are executed automatically. 
* Failure to comply with the following instructions will result in a significant penalty. 
* Appeals regarding your failure to read these instructions will be denied. 
* Kind reminder: the homework assignments contribute 60% of the final grade.


## Read the following instructions carefully:

1. This Jupyter notebook contains all the step-by-step instructions needed for this exercise.
1. Write **efficient**, **vectorized** code whenever possible. Some calculations in this exercise may take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deductions.
1. You are responsible for the correctness of your code and should add as many tests as you see fit to this jupyter notebook. Tests will not be graded nor checked.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/).
1. Your code must run without errors. Use at least `numpy` 1.15.4. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Submission includes a zip file that contains this notebook, with your ID as the file name. For example, `hw1_123456789_987654321.zip` if you submitted in pairs and `hw1_123456789.zip` if you submitted the exercise alone. The name of the notebook should follow the same structure.
   
Please use only a **zip** file in your submission.

---
---

## Please sign that you have read and understood the instructions: 

### *** YOUR RUNI EMAILS HERE ***
shahar.shvili@post.runi.ac.il,itay.baror@post.runi.ac.il
---
---


In [12]:
# Import necessary libraries
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

np.random.seed(42)

# Design your algorithm
Make sure to describe the algorithm, its limitations, and describe use-cases.

**Overview**

when using Adaboost we are given feature vectors 
$x_i \in \mathbb{R}^d$ 
with binary lables 
$y_i \in \{-1, +1\}$ 
our goal is to combine many weak learners classifiers into a strong classifier with low traning error. the algorithm achieves this by iterating over the stumps and adding the ones that make the model better and adjusting the weights of the samples according to if they are missclassified or not 

**Algorithem**

let n be the number of samples, and T the number of boosting rounds

**Step 1: Initialization**

initialize sample weights for every 
$i=1,...,n$ $w_i=1/n$ 
at the beginning all samples  are equally as important as we continue the weights will change acording to if we misclassifed the samples
initialize the ensemble prediction function 
$H_0=0$ 
we start with no classifier while we build the model we will add stumps to here 

**Step 2:**

For t=0,....,T-1
each iteration adds one weak learner and updates the weights 

**Step 2.1: Train the weak learner**

Train the weak leaner using the current weights. this means that samples with larger weights have more influence on the traning. 

**Step 2.2: Compute weighted classification error**

$\epsilon = \sum_{i : h(x_i) \neq y_i} w_i$

calculate how good the classification did in the weighted dataset  
- if $\epsilon<=0.5$ then we got to the end since our current weak learner does not add to our classification 
- if not continue to the next step 

**Step 2.3: Compute the weight of the weak learner**

$\alpha = \frac{1}{2} \ln\!\left(\frac{1-\epsilon}{\epsilon}\right)$

when weak learners have low error then we want them to matter more so there weight will be large 

**Step 2.4: Update the model with the current weak learner**

add the current weak learner to the model of all the weak learners we add up until now 
$H_{t+1} = H_t + \alpha h$

**Step 2.5: Update the weights**
update the weights of samples that are currently in the model 

$w_i \leftarrow \frac{w_i e^{-\alpha h(x_i) y_i}}{2\sqrt{\epsilon (1-\epsilon)}}$

misclassified samples get higher weights correctly classified ones get lower weights 

**Step 3: return the final model**

return $H(x) = \operatorname{sign}\!\left( \sum_{t=1}^T \alpha_t h_t(x) \right)$


# Your implementations
You may add new cells, write helper functions or test code as you see fit.
Please use the cell below and include a description of your implementation.
Explain code design consideration, algorithmic choices and any other details you think is relevant to understanding your implementation.
Failing to explain your code will lead to point deductions.

In [13]:
# import numpy as np
# from sklearn.tree import DecisionTreeClassifier


# # Part 1: Implementing AdaBoost
# class AdaBoostCustom:
#     def __init__(self, T):
#         self.T = T
#         self.alphas = []
#         self.models = []
#         self.classes = None
#         # Note: You may add more attributes

#     def fit(self, X, y):
#         # Your code here
#         pass
    
#     def predict(self, X):  
#         # Your code here
#         pass



import numpy as np
from sklearn.tree import DecisionTreeClassifier


class AdaBoostCustom:
    def __init__(self, T=100, base_estimator=None, learning_rate=1.0, random_state=None):
        self.T = int(T)
        self.learning_rate = float(learning_rate)
        self.random_state = random_state
        self.base_estimator = base_estimator or DecisionTreeClassifier(
            max_depth=1, random_state=random_state
        )
        self.alphas = []
        self.models = []
        self.neg_class = None
        self.pos_class = None

    # Encode to {-1,+1} by picking a deterministic positive/negative class.
    def encode_pm1(self, y):
        y = np.asarray(y)
        classes = np.unique(y)
        if classes.size != 2:
            raise ValueError("AdaBoostCustom supports binary classification only.")
        self.neg_class, self.pos_class = classes[0], classes[1]
        return np.where(y == self.pos_class, 1, -1)

    # Decode {-1,+1} back to original labels.
    def decode_labels(self, y_pm1):
        return np.where(y_pm1 >= 0, self.pos_class, self.neg_class)

    # Core boosting loop with weighted trees, SAMME alpha, and weight renormalization.
    def fit(self, X, y):
        X = np.asarray(X)
        y_pm1 = self.encode_pm1(y).astype(float)

        n = X.shape[0]
        D = np.full(n, 1.0 / n)
        eps = 1e-12

        self.alphas = []
        self.models = []

        for t in range(self.T):
            stump = DecisionTreeClassifier(**self.base_estimator.get_params())
            stump.fit(X, y_pm1, sample_weight=D)

            h_pm1 = stump.predict(X).astype(float)
            err = float(np.sum(D * (h_pm1 != y_pm1)))
            err = float(np.clip(err, 0.0, 1.0))

            if err >= 0.5 - 1e-15:
                if t == 0:
                    self.alphas.append(1e-6)
                    self.models.append(stump)
                break

            alpha = 0.5 * np.log((1.0 - err) / (err + eps))
            alpha *= self.learning_rate

            D *= np.exp(-alpha * y_pm1 * h_pm1)
            s = D.sum()
            if s == 0.0:
                self.alphas.append(alpha)
                self.models.append(stump)
                break
            D /= s

            self.alphas.append(alpha)
            self.models.append(stump)

        return self

    # Margin F(x) = sum_t alpha_t h_t(x).
    def decision_function(self, X):
        if not self.models:
            raise ValueError("Model is not fitted.")
        X = np.asarray(X)
        F = np.zeros(X.shape[0], dtype=float)
        for alpha, stump in zip(self.alphas, self.models):
            F += alpha * stump.predict(X).astype(float)
        return F

    # Final classifier H(x) = sign(F(x)); ties go to +1.
    def predict(self, X):
        F = self.decision_function(X)
        y_pm1 = np.sign(F)
        y_pm1[y_pm1 == 0] = 1
        return self.decode_labels(y_pm1.astype(int))

# Generate data
Please use the cell below to discuss your dataset choice and why it is appropriate (or not) for this algorithm.

In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def simple_dataset(n=800, class_sep=2.0, test_size=0.3, random_state=0):
    X, y = make_classification(
        n_samples=n, n_features=2, n_informative=2, n_redundant=0,
        n_clusters_per_class=1, class_sep=class_sep, flip_y=0.0,
        random_state=random_state
    )
    return train_test_split(X, y, test_size=test_size, stratify=y, random_state=random_state)


# --- generate separable dataset ---
X_train, X_test, y_train, y_test = simple_dataset(n=800, class_sep=2.0, test_size=0.3, random_state=0)



# AdaBoost demonstration 
Demonstrate your AdaBoost implementation.

Add plots and figures. 

Please use the cell below to describe your results and tests.

Describe the difference between your implementation and the sklearn implementation. Hint: you can look at the documentation.

In [15]:
custom_model = AdaBoostCustom(T=10)
sklearn_model = AdaBoostClassifier(n_estimators=10, random_state=42)

# Your code here
custom_model.fit(X_train, y_train)
sklearn_model.fit(X_train, y_train) 
# Predictions
y_pred_custom = custom_model.predict(X_test)
y_pred_sklearn = sklearn_model.predict(X_test)
# Accuracy
acc_custom = accuracy_score(y_test, y_pred_custom)
acc_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Custom AdaBoost Accuracy: {acc_custom:.4f}")
print(f"Sklearn AdaBoost Accuracy: {acc_sklearn:.4f}")


Custom AdaBoost Accuracy: 0.9917
Sklearn AdaBoost Accuracy: 0.9917


# Generate additional data sets
Generate at least two experimental datasets with binary labels, designed to demonstrate specific properties of AdaBoost (e.g., handling noise or overfitting).

Add plots and figures.

Please use the cell below to describe your suggested approach in detail. Use formal notations where appropriate.

Describe and discuss your results.

In [16]:
# Generate additional data sets

# Split data sets

In [17]:
# Visualize data

# Test algorithms
Test your AdaBoost, a library implementation of AdaBoost and at least two additional models, one of which must be another boosting algorithm on your two datasets.

Add plots and figures.

Please use the cell below to describe your suggested approach in detail. Use formal notations where appropriate.

Describe and discuss your results.

In [18]:
# Set up models 

In [19]:
# Test and visualize

# Use of generative AI
Please use the cell below to describe your use of generative AI in this assignment. 